In [ ]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras.callbacks import ReduceLROnPlateau, Callback, ModelCheckpoint
from sklearn.metrics import cohen_kappa_score
from keras.utils import np_utils
from keras import backend as K
from keras import regularizers
from keras.optimizers import Adam
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.applications import DenseNet121
from tqdm import tqdm
from sklearn.utils import class_weight


print(os.listdir("../input/densenet"))
densenet = DenseNet121(
    weights='../input/densenet/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5',
    include_top=False,
    input_shape=(224,224,3)
)

In [ ]:
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        X_val, Y_val = self.validation_data[:2]
        Y_val = Y_val.sum(axis=1) - 1
        
        Y_pred = self.model.predict(X_val) > 0.5
        Y_pred = Y_pred.astype(int).sum(axis=1) - 1

        _val_kappa = cohen_kappa_score(
            Y_val,
            Y_pred, 
            weights='quadratic'
        )

        self.val_kappas.append(_val_kappa)

        print(f"val_kappa: {_val_kappa:.4f}")
        
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model.")
            self.model.save('model.h5')

        return

In [ ]:
model = Sequential()
model.add(densenet)
model.add(GlobalAveragePooling2D(
))
model.add(Dropout(0.5))
model.add(Dense(5, activation='sigmoid'))#, kernel_regularizer=regularizers.l2(0.0001)
               #,activity_regularizer=regularizers.l1(0.01)))
model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])
mc = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only = True, mode ='min', verbose = 1)
rl = reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, mode='auto', cooldown=1, min_lr=0.000005)
qwk = Metrics()

In [ ]:
for layer in model.layers:
    layer.trainable = False

for i in range(-3,0):
    model.layers[i].trainable = True

In [ ]:
df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')

x_train = []
y_train = []

for image_name in df['id_code']:
    temp_label = df.loc[df.id_code == image_name, 'diagnosis'].values[0]
    image = cv2.imread('../input/aptos2019-blindness-detection/train_images/'+image_name+'.png')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    mask = gray_img>7
    img1=image[:,:,0][np.ix_(mask.any(1),mask.any(0))]
    img2=image[:,:,1][np.ix_(mask.any(1),mask.any(0))]
    img3=image[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    img = np.stack([img1,img2,img3],axis=-1)
    image_arr = cv2.resize(img, (224, 224), interpolation = cv2.INTER_LANCZOS4)
    
    #image_arr = cv2.resize(image, (224, 224), interpolation = cv2.INTER_LANCZOS4)
    
    image_arr = cv2.addWeighted (image_arr, 4, cv2.GaussianBlur(image_arr, (0, 0), 10), -4, 128)
    image_arr = image_arr.astype('float32')
    image_arr /= 255
    x_train.append(image_arr)
    y_train.append(temp_label)

x_train = np.array(x_train)
x_train = x_train.reshape(x_train.shape[0], 224, 224, 3)
y_train = np.array(y_train)
x_train = x_train.astype('float32')
input_shape = (224, 224, 3)

In [ ]:
y_train = np_utils.to_categorical(y_train) 

y_train1 = np.zeros(y_train.shape, dtype=y_train.dtype)
for x in range(4, -1, -1):
    for i in range(x, -1, -1):
        y_train1[:,i] += y_train[:,x]
y_train1 = np.array(y_train1)
y_train = y_train1

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

In [ ]:
y_integers = []

for label in y_train:
    for e in range(len(label)):
        if label[e] == 1:
            y_integers.append(e)

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_integers),
                                                 y_integers)

In [ ]:
datagen = ImageDataGenerator(
    zoom_range=0.2,
    fill_mode='constant',
    cval=0.,
    horizontal_flip=True,
    vertical_flip=True)

In [ ]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=32), validation_data=(x_val, y_val),
                    steps_per_epoch=(len(x_train)/32), epochs=2)#, class_weight=class_weights)

In [ ]:
for layer in model.layers:
    layer.trainable = True

In [ ]:
model.compile(optimizer=Adam(lr=0.00005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=32), validation_data=(x_val, y_val),
                    steps_per_epoch=(len(x_train)/32), epochs=60, callbacks=[qwk])#, class_weight=class_weights, callbacks=[rl, qwk])

In [ ]:
densenet = None
x_train = None
x_train1 = None
y_train = None
y_train1 = None
x_val = None
y_val = None

del densenet
del x_train
del x_train1
del y_train
del y_train1
del x_val
del y_val
K.clear_session()

In [ ]:
model = load_model('model.h5')
model.get_weights()
model.optimizer

submission_df = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv') 
predictions = []
for i, name in tqdm(enumerate(submission_df['id_code'])):
    path = os.path.join('../input/aptos2019-blindness-detection/test_images/', name+'.png')
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mask = gray_img>7
    img1=image[:,:,0][np.ix_(mask.any(1),mask.any(0))]
    img2=image[:,:,1][np.ix_(mask.any(1),mask.any(0))]
    img3=image[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    img = np.stack([img1,img2,img3],axis=-1)
    image = cv2.resize(img, (224, 224), interpolation = cv2.INTER_LANCZOS4)
    
    #image = cv2.resize(image, (224, 224), interpolation = cv2.INTER_LANCZOS4)
    
    image = cv2.addWeighted (image, 4, cv2.GaussianBlur(image, (0, 0), 10), -4, 128)
    image = image.astype('float32')
    image /= 255
    image = image.reshape(1, 224, 224, 3)
    score_predict=((model.predict(image).ravel()+model.predict(image[:, ::-1, :, :]).ravel()+model.predict(image[:, ::-1, ::-1, :]).ravel()+model.predict(image[:, :, ::-1, :]).ravel())*0.25).tolist()
    #score_predict=((model.predict(image).ravel()*model.predict(image[:, ::-1, :, :]).ravel()*model.predict(image[:, ::-1, ::-1, :]).ravel()*model.predict(image[:, :, ::-1, :]).ravel())**0.25).tolist()
    score_predict = np.array(score_predict) > 0.5
    label_predict = score_predict.astype(int).sum() - 1
    predictions.append(str(label_predict))

submission_df['diagnosis'] = predictions
submission_df.to_csv('submission.csv', index=False)